In [28]:
import csv
import os
from collections import Counter

import pandas
from pandas import DataFrame
import requests
from datasets import load_dataset, Dataset
from datasets import ClassLabel
import re

In [40]:
label_feature = ClassLabel(names=['entailment', 'neutral', 'contradiction'])

def convert_labels_to_classlabel(example):
    # Konvertiere den int64 label in den entsprechenden String-Wert
    example['label'] = label_feature.int2str(example['label'])
    return example

In [30]:
xnli = load_dataset(path="xnli",
                    name="de")
article_key_name = "premise"
sentence_key_name = "hypothesis"
label_key_name = "label"
text_len_count_key_name = "num_words_article_and_sentence"
min_text_length = 50
max_text_length = 500
seed = 42
sub_sample_size = 3000

Resolving data files: 100%|██████████| 19/19 [00:00<00:00, 113.75it/s]


In [31]:
train_df = xnli["train"].to_pandas()
train_df[text_len_count_key_name] = train_df[article_key_name].str.len() + train_df[sentence_key_name].str.len()

In [32]:
filtered_train_df = train_df[
    (train_df[text_len_count_key_name] > min_text_length) & (
            train_df[text_len_count_key_name] < max_text_length)]
sub_sampled_train_df = filtered_train_df.sample(n=sub_sample_size, random_state=seed)
sub_sampled_train_df.drop(columns=[text_len_count_key_name], inplace=True)
print(sub_sampled_train_df[label_key_name].value_counts())

label
1    1031
0     988
2     981
Name: count, dtype: int64


In [41]:
subsampled_train_dataset = Dataset.from_pandas(sub_sampled_train_df.reset_index(drop=True))

# Wenden Sie die Konvertierungsfunktion auf den 'train'-Teil des DatasetDict an
subsampled_train_dataset= subsampled_train_dataset.map(convert_labels_to_classlabel)
subsampled_train_dataset = subsampled_train_dataset.cast_column('label', label_feature)
subsampled_train_dataset.info
# Überprüfen Sie, ob die 

Casting the dataset: 100%|██████████| 3000/3000 [00:00<00:00, 678982.95 examples/s]


DatasetInfo(description='', citation='', homepage='', license='', features={'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [42]:
xnli["train"] = subsampled_train_dataset
xnli.push_to_hub("mtc/xnli_de_sub_sampled_3000")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


In [43]:
val = xnli["validation"].to_pandas()
val["label"].value_counts()

label
1    830
2    830
0    830
Name: count, dtype: int64

In [44]:
test = xnli["test"].to_pandas()
test["label"].value_counts()

label
2    1670
0    1670
1    1670
Name: count, dtype: int64